# 🚀 Entrenamiento de Red Neuronal CNN para Super-Resolución de Señales

## 📋 Descripción
Este notebook te guiará paso a paso para entrenar una red neuronal convolucional (CNN) que convierte señales de **baja resolución** a **alta resolución**.

### ¿Qué hace este modelo?
- **Entrada**: Señal con pocos puntos (ejemplo: 1000 puntos)
- **Salida**: Señal con muchos puntos (ejemplo: 5000 puntos)
- **Aplicación**: Reconstrucción de señales ECG, señales de audio, series temporales, etc.

---

## 📦 1. Instalación de Librerías

### Librerías necesarias:
- **PyTorch**: Framework de deep learning
- **NumPy**: Manejo de arrays y matrices
- **Matplotlib**: Visualización de gráficos
- **tqdm**: Barras de progreso
- **temana**: Librería personalizada para lectura de datos (debe estar en tu carpeta)

### Instalación:
Si no tienes las librerías instaladas, ejecuta esta celda (descomenta las líneas):

In [ ]:
# Descomenta y ejecuta si necesitas instalar las librerías
# !pip install torch torchvision
# !pip install numpy matplotlib tqdm

---
## 📚 2. Importar Librerías

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import random

# Librería personalizada (asegúrate de tener temana.py en tu carpeta)
import temana as tm

print("✅ Librerías importadas correctamente")
print(f"PyTorch versión: {torch.__version__}")

---
## 🏗️ 3. Definición del Modelo

### ¿Qué es TimeSeriesSRNet?
Es una red neuronal convolucional con:
- **Encoder**: Extrae características de la señal de entrada
- **Upsampler**: Aumenta la resolución de la señal

### Parámetros:
- `upsample_factor`: Factor de aumento (5 = convierte 1000 puntos → 5000 puntos)

In [ ]:
class TimeSeriesSRNet(nn.Module):
    def __init__(self, upsample_factor=5):
        super(TimeSeriesSRNet, self).__init__()
        
        # Encoder: Extrae características de la señal
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=9, stride=1, padding=4),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv1d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
        )
        
        # Upsampler: Aumenta la resolución
        self.upsample = nn.Sequential(
            nn.Upsample(scale_factor=upsample_factor, mode='linear', align_corners=True),
            nn.Conv1d(256, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv1d(128, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Conv1d(64, 1, kernel_size=9, stride=1, padding=4)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.upsample(x)
        return x

print("✅ Modelo definido correctamente")

---
## 📂 4. Carga de Datos

### Formato de los datos:
Los datos deben estar en archivos `.txt` con el siguiente formato:
- **Cada fila** = Una señal completa
- **Cada columna** = Un punto de la señal

Ejemplo:
```
0.5  0.6  0.7  ...  (1000 columnas para baja resolución)
0.3  0.4  0.5  ...  (otra señal)
```

### Archivos necesarios:
1. **x_train**: Señales de baja resolución (entrada)
2. **y_train**: Señales de alta resolución (objetivo)

### ⚠️ Importante:
- Las señales `x_train` y `y_train` deben tener **el mismo número de filas** (mismo número de muestras)
- `x_train` debe tener menos columnas que `y_train` (menor resolución)

In [ ]:
# Rutas de tus archivos de datos
# CAMBIA ESTAS RUTAS según donde tengas tus datos

x_train_path = 'Samples/SamplesAV_FV2024_07_09/SignalAVFV_Sub_Sample.txt'
y_train_path = 'Samples/SamplesAV_FV2024_07_09/SignalAVFV_Super_Sample.txt'

# Cargar datos usando temana
x_train = tm.read_data(x_train_path)
y_train = tm.read_data(y_train_path)

print(f"✅ Datos cargados correctamente")
print(f"📊 x_train shape: {x_train.shape} (Muestras, Puntos)")
print(f"📊 y_train shape: {y_train.shape} (Muestras, Puntos)")
print(f"📈 Factor de aumento: {y_train.shape[1] // x_train.shape[1]}x")

---
## 👀 5. Visualización de los Datos

Veamos cómo se ven las señales de baja y alta resolución:

In [ ]:
# Seleccionar una muestra aleatoria
sample_idx = np.random.randint(0, x_train.shape[0])

# Crear dominio temporal
tx_train = np.linspace(0, 4*np.pi, x_train.shape[1])
ty_train = np.linspace(0, 4*np.pi, y_train.shape[1])

# Graficar
plt.figure(figsize=(14, 5))
plt.plot(tx_train, x_train[sample_idx, :], 'o-', label='Baja Resolución (Entrada)', markersize=4)
plt.plot(ty_train, y_train[sample_idx, :], '-', label='Alta Resolución (Objetivo)', linewidth=2)
plt.title(f'Ejemplo de Señal - Muestra #{sample_idx}')
plt.xlabel('Tiempo')
plt.ylabel('Amplitud')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"💡 La señal de baja resolución tiene {x_train.shape[1]} puntos")
print(f"💡 La señal de alta resolución tiene {y_train.shape[1]} puntos")

---
## ⚙️ 6. Configuración de Hiperparámetros

### Hiperparámetros explicados:
- **low_res_points**: Número de puntos en señal de entrada
- **high_res_points**: Número de puntos en señal de salida
- **upsample_factor**: Factor de aumento (debe ser high_res / low_res)
- **epochs**: Número de veces que el modelo verá todos los datos
- **batch_size**: Número de señales procesadas simultáneamente
- **learning_rate**: Velocidad de aprendizaje (típicamente entre 0.0001 y 0.01)

In [ ]:
# Configuración automática basada en los datos
low_res_points = x_train.shape[1]
high_res_points = y_train.shape[1]
upsample_factor = high_res_points // low_res_points

# Hiperparámetros de entrenamiento
epochs = 100           # Puedes aumentar para mejor resultado (pero toma más tiempo)
batch_size = 32        # Reduce si tienes poca memoria RAM
learning_rate = 1e-3   # 0.001

print("⚙️ Configuración:")
print(f"  - Puntos entrada: {low_res_points}")
print(f"  - Puntos salida: {high_res_points}")
print(f"  - Factor de aumento: {upsample_factor}x")
print(f"  - Épocas: {epochs}")
print(f"  - Batch size: {batch_size}")
print(f"  - Learning rate: {learning_rate}")

---
## 🎯 7. Inicialización del Modelo

Creamos el modelo, el optimizador y la función de pérdida:

In [ ]:
# Crear el modelo
model = TimeSeriesSRNet(upsample_factor=upsample_factor)

# Optimizador (Adam es el más usado)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Función de pérdida (L1 Loss = Mean Absolute Error)
criterion = nn.L1Loss()

# Contar parámetros del modelo
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("✅ Modelo inicializado")
print(f"📊 Parámetros totales: {total_params:,}")
print(f"🎓 Parámetros entrenables: {trainable_params:,}")

---
## 🏋️ 8. Función de Entrenamiento

Esta función entrena el modelo durante varias épocas:

In [ ]:
def train(model, optimizer, criterion, epochs, batch_size, x_train, y_train):
    """
    Entrena el modelo de super-resolución.
    
    Args:
        model: Modelo de red neuronal
        optimizer: Optimizador (Adam, SGD, etc.)
        criterion: Función de pérdida
        epochs: Número de épocas de entrenamiento
        batch_size: Tamaño del batch
        x_train: Datos de entrada (baja resolución)
        y_train: Datos objetivo (alta resolución)
    """
    model.train()
    dataset_size = x_train.shape[0]
    indices = np.arange(dataset_size)
    
    # Lista para guardar el historial de pérdidas
    loss_history = []
    
    print("🚀 Iniciando entrenamiento...\n")
    
    for epoch in range(epochs):
        # Mezclar los datos al inicio de cada época
        np.random.shuffle(indices)
        epoch_loss = 0.0
        
        # Procesar por batches
        for start_idx in range(0, dataset_size, batch_size):
            end_idx = min(start_idx + batch_size, dataset_size)
            batch_idx = indices[start_idx:end_idx]
            
            # Preparar batch y convertir a tensores
            x_batch = torch.tensor(x_train[batch_idx][:, np.newaxis, :], dtype=torch.float32)
            y_batch = torch.tensor(y_train[batch_idx][:, np.newaxis, :], dtype=torch.float32)
            
            # Forward pass
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item() * x_batch.size(0)
        
        # Calcular pérdida promedio de la época
        avg_loss = epoch_loss / dataset_size
        loss_history.append(avg_loss)
        
        # Mostrar progreso cada 10 épocas
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"Época [{epoch+1}/{epochs}] - Loss: {avg_loss:.6f}")
    
    print("\n✅ Entrenamiento completado")
    return loss_history

print("✅ Función de entrenamiento definida")

---
## 🎓 9. Entrenar el Modelo

¡Ahora sí! Vamos a entrenar el modelo. **Esto puede tomar varios minutos**:

In [ ]:
# Entrenar el modelo
loss_history = train(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    epochs=epochs,
    batch_size=batch_size,
    x_train=x_train,
    y_train=y_train
)

print(f"\n🎉 Entrenamiento finalizado con éxito")
print(f"📉 Loss final: {loss_history[-1]:.6f}")

---
## 📈 10. Visualizar Curva de Aprendizaje

Veamos cómo mejoró el modelo durante el entrenamiento:

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(loss_history, linewidth=2)
plt.title('Curva de Aprendizaje', fontsize=14, fontweight='bold')
plt.xlabel('Época')
plt.ylabel('Loss (L1)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"💡 El modelo comenzó con loss = {loss_history[0]:.6f}")
print(f"💡 El modelo terminó con loss = {loss_history[-1]:.6f}")
print(f"📊 Mejora: {((loss_history[0] - loss_history[-1]) / loss_history[0] * 100):.1f}%")

---
## 💾 11. Guardar el Modelo

Guardamos el modelo entrenado para usarlo después:

In [ ]:
# Crear carpeta Models si no existe
os.makedirs('Models', exist_ok=True)

# Nombre del archivo
model_name = f"timeseries_srnet_{low_res_points}_to_{high_res_points}.pth"
model_path = os.path.join('Models', model_name)

# Guardar el modelo
torch.save(model.state_dict(), model_path)

print(f"✅ Modelo guardado en: {model_path}")
print(f"📦 Tamaño del archivo: {os.path.getsize(model_path) / (1024**2):.2f} MB")

---
## 🔍 12. Funciones de Evaluación

Definimos funciones para evaluar el modelo:

In [ ]:
def calculate_l1_distance(model, x, y_true):
    """
    Calcula la distancia L1 (error absoluto medio) entre la señal real y la predicción.
    """
    model.eval()
    x_tensor = torch.tensor(x[np.newaxis, np.newaxis, :], dtype=torch.float32)
    y_true = np.array(y_true)
    
    with torch.no_grad():
        y_pred = model(x_tensor).squeeze().numpy()
    
    l1_distance = np.mean(np.abs(y_true - y_pred))
    return l1_distance


def mean_l1_loss(model, x_val, y_val):
    """
    Calcula el loss L1 promedio sobre un conjunto de validación.
    """
    model.eval()
    total_loss = 0.0
    
    for i in range(x_val.shape[0]):
        x = x_val[i]
        y_true = y_val[i]
        loss = calculate_l1_distance(model, x, y_true)
        total_loss += loss
    
    mean_loss = total_loss / x_val.shape[0]
    return mean_loss

print("✅ Funciones de evaluación definidas")

---
## 📊 13. Visualizar Predicciones

Veamos qué tan bien funciona el modelo entrenado:

In [ ]:
def plot_predictions(model, x_data, y_data, num_samples=4):
    """
    Grafica predicciones del modelo vs señales reales.
    """
    model.eval()
    indices = random.sample(range(x_data.shape[0]), num_samples)
    
    plt.figure(figsize=(16, 3 * num_samples))
    
    for i, idx in enumerate(indices):
        x = torch.tensor(x_data[idx][np.newaxis, np.newaxis, :], dtype=torch.float32)
        y_true = y_data[idx]
        
        with torch.no_grad():
            y_pred = model(x).squeeze().numpy()
        
        # Calcular error
        l1_error = calculate_l1_distance(model, x_data[idx], y_true)
        
        # Graficar
        plt.subplot(num_samples, 1, i + 1)
        plt.plot(y_true, label='Real (Alta Res)', linewidth=2, alpha=0.7)
        plt.plot(y_pred, label='Predicción', linestyle='--', linewidth=2)
        plt.scatter(np.linspace(0, len(y_true)-1, len(x_data[idx])), 
                   np.interp(np.linspace(0, len(y_true)-1, len(x_data[idx])), 
                            np.arange(len(y_true)), y_true),
                   color='red', label='Entrada (Baja Res)', s=20, zorder=5)
        plt.title(f'Muestra {idx} - Error L1: {l1_error:.4f}')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Visualizar predicciones en datos de entrenamiento
print("🎨 Predicciones del modelo en datos de entrenamiento:\n")
plot_predictions(model, x_train, y_train, num_samples=4)

---
## 📈 14. Calcular Métricas en Conjunto de Entrenamiento

In [ ]:
# Calcular error promedio en datos de entrenamiento
train_loss = mean_l1_loss(model, x_train, y_train)

print("📊 Métricas en Conjunto de Entrenamiento:")
print(f"  - Loss L1 promedio: {train_loss:.6f}")
print(f"  - Número de muestras: {x_train.shape[0]}")

---
## 🔄 15. OPCIONAL: Cargar Datos de Validación

Si tienes un conjunto de validación separado, cárgalo aquí:

In [ ]:
# Descomenta y modifica estas líneas si tienes datos de validación

# x_val_path = 'Samples/SamplesAV_FV2024_07_09/SignalAVFV_Sub_Sample_Val.txt'
# y_val_path = 'Samples/SamplesAV_FV2024_07_09/SignalAVFV_Super_Sample_Val.txt'

# x_val = tm.read_data(x_val_path)
# y_val = tm.read_data(y_val_path)

# print(f"✅ Datos de validación cargados")
# print(f"📊 x_val shape: {x_val.shape}")
# print(f"📊 y_val shape: {y_val.shape}")

# # Calcular métricas en validación
# val_loss = mean_l1_loss(model, x_val, y_val)
# print(f"\n📊 Loss en validación: {val_loss:.6f}")

# # Visualizar predicciones en validación
# plot_predictions(model, x_val, y_val, num_samples=4)

print("ℹ️ Sección de validación opcional (descomenta para usar)")

---
## 🔄 16. Transfer Learning: Cargar Modelo Pre-entrenado

### ¿Qué es Transfer Learning?
Es cuando cargas un modelo ya entrenado y lo entrenas un poco más con datos nuevos.

**Útil cuando:**
- Tienes un modelo entrenado con datos sintéticos y quieres mejorarlo con datos reales
- Quieres continuar el entrenamiento desde donde lo dejaste

In [ ]:
def load_model_for_transfer_learning(model_path, upsample_factor):
    """
    Carga un modelo pre-entrenado para transfer learning.
    """
    model = TimeSeriesSRNet(upsample_factor=upsample_factor)
    model.load_state_dict(torch.load(model_path))
    model.train()  # Poner en modo entrenamiento
    print(f"✅ Modelo cargado desde: {model_path}")
    return model

# Ejemplo de uso (descomenta para usar):
# model_pretrained = load_model_for_transfer_learning('Models/timeseries_srnet_1000_to_5000.pth', upsample_factor=5)

# # Cargar nuevos datos (por ejemplo, datos reales)
# x_real = tm.read_data('RealSamples/ecg_low_5000.txt')
# y_real = tm.read_data('RealSamples/ecg_high_5000.txt')

# # Escalar datos si es necesario
# x_real_scaled = x_real / 40.0
# y_real_scaled = y_real / 40.0

# # Nuevo optimizador (puedes usar learning rate más bajo)
# optimizer_transfer = optim.Adam(model_pretrained.parameters(), lr=1e-4)

# # Entrenar con menos épocas
# loss_history_transfer = train(
#     model=model_pretrained,
#     optimizer=optimizer_transfer,
#     criterion=criterion,
#     epochs=20,
#     batch_size=32,
#     x_train=x_real_scaled,
#     y_train=y_real_scaled
# )

# # Guardar modelo ajustado
# torch.save(model_pretrained.state_dict(), 'Models/timeseries_srnet_transfer_learned.pth')

print("ℹ️ Sección de transfer learning (descomenta para usar)")

---
## 🔧 17. Troubleshooting Común

### ❌ Problema: "Out of Memory"
**Solución:**
- Reduce `batch_size` (ejemplo: de 32 a 16 o 8)
- Si usas GPU, libera memoria: `torch.cuda.empty_cache()`

### ❌ Problema: El loss no baja
**Posibles causas:**
- Learning rate muy alto → Prueba con `1e-4` en lugar de `1e-3`
- Learning rate muy bajo → Prueba con `1e-2`
- Datos mal escalados → Normaliza tus datos
- Necesitas más épocas → Aumenta `epochs`

### ❌ Problema: "FileNotFoundError"
**Solución:**
- Verifica que las rutas de los archivos sean correctas
- Usa rutas absolutas si tienes problemas con rutas relativas

### ❌ Problema: Las predicciones son malas
**Soluciones:**
1. Entrena por más épocas
2. Aumenta el tamaño del dataset
3. Verifica que los datos de entrada y salida estén bien alineados
4. Prueba con transfer learning si tienes pocos datos

---
## 📝 18. Resumen y Próximos Pasos

### ✅ Lo que hicimos:
1. ✅ Instalamos y cargamos librerías
2. ✅ Definimos el modelo CNN para super-resolución
3. ✅ Cargamos y visualizamos los datos
4. ✅ Entrenamos el modelo
5. ✅ Evaluamos el rendimiento
6. ✅ Guardamos el modelo

### 🚀 Próximos pasos sugeridos:
1. **Experimentar con hiperparámetros:**
   - Prueba diferentes `learning_rate`
   - Aumenta `epochs` para mejor resultado
   - Experimenta con diferentes `batch_size`

2. **Mejorar el modelo:**
   - Usa transfer learning con datos reales
   - Prueba con diferentes arquitecturas
   - Implementa data augmentation

3. **Validación:**
   - Crea un conjunto de validación separado
   - Usa cross-validation
   - Compara con otros métodos (interpolación lineal, splines)

4. **Aplicación:**
   - Usa el modelo entrenado en datos nuevos
   - Crea una función de inferencia
   - Exporta el modelo para producción

### 📚 Recursos adicionales:
- [Documentación de PyTorch](https://pytorch.org/docs/stable/index.html)
- [Tutorial de CNNs](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)
- [Transfer Learning en PyTorch](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

---
## 💡 19. Guardar y Cargar Modelos - Referencia Rápida

In [ ]:
# === GUARDAR MODELO ===
# torch.save(model.state_dict(), 'ruta/del/modelo.pth')

# === CARGAR MODELO PARA INFERENCIA ===
# model = TimeSeriesSRNet(upsample_factor=5)
# model.load_state_dict(torch.load('ruta/del/modelo.pth'))
# model.eval()  # Modo evaluación

# === CARGAR MODELO PARA CONTINUAR ENTRENAMIENTO ===
# model = TimeSeriesSRNet(upsample_factor=5)
# model.load_state_dict(torch.load('ruta/del/modelo.pth'))
# model.train()  # Modo entrenamiento
# optimizer = optim.Adam(model.parameters(), lr=1e-4)

print("📖 Referencia rápida de guardado/carga de modelos")

---
## 🎉 ¡Felicidades!

Has completado el entrenamiento de tu modelo de super-resolución de señales.

**Creado por:** GitHub Copilot con Claude Sonnet 4.5  
**Fecha:** Noviembre 2025